# Perky Penguins - Climate Change Data Scraping and Cleaning

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager
import pathlib

In [2]:
!pip install pytrends

In [3]:
from pytrends.request import TrendReq
pytrend = TrendReq()

### Import and clean CO2 Emission & Population data from The World Bank. The result will be used for visualization.

In [4]:
# Name the CO2 Emission CSV file
CO2_emission_file = pathlib.Path('CO2_emissions_per_capita.csv')

# Read and display the CO2 Emission CSV with Pandas
CO2_emission_df = pd.read_csv(CO2_emission_file)

In [158]:
CO2_emission_df_2 = CO2_emission_df[['Country Name','Country Code', 'Indicator Name', '2006', '2007','2008','2009','2010','2011','2012','2013','2014','2015','2016']]
CO2_emission_df_2

,Country Name,Country Code,Indicator Name,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,CO2 emissions (metric tons per capita),26.947726,27.895023,26.229553,25.915322,24.670529,24.507516,13.157722,8.353561,8.410064,8.610374,8.426911
1,Afghanistan,AFG,CO2 emissions (metric tons per capita),0.062428,0.083893,0.151721,0.238399,0.289988,0.406424,0.345149,0.280455,0.253728,0.262556,0.245101
2,Angola,AGO,CO2 emissions (metric tons per capita),1.105019,1.203134,1.185000,1.234425,1.244092,1.262825,1.361181,1.295075,1.664741,1.240245,1.202861
3,Albania,ALB,CO2 emissions (metric tons per capita),1.302576,1.322335,1.484311,1.495600,1.578574,1.803715,1.697966,1.697279,1.900070,1.602648,1.577163
4,Andorra,AND,CO2 emissions (metric tons per capita),6.746052,6.519387,6.427810,6.121580,6.122595,5.867410,5.916884,5.901775,5.832906,5.969786,6.072370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,CO2 emissions (metric tons per capita),NaN,NaN,4.266386,4.650695,4.745656,4.672303,4.395272,4.441262,4.005283,4.702186,5.031520
260,"Yemen, Rep.",YEM,CO2 emissions (metric tons per capita),1.030386,0.985390,1.020428,1.090990,1.011975,0.865783,0.764021,1.013754,0.955251,0.497091,0.390480
261,South Africa,ZAF,CO2 emissions (metric tons per capita),9.236950,9.498786,9.979458,9.967159,9.173055,9.179457,8.794997,8.553336,8.882322,8.376655,8.480658
262,Zambia,ZMB,CO2 emissions (metric tons per capita),0.188267,0.154271,0.168958,0.189800,0.197823,0.208412,0.254267,0.265078,0.292412,0.285428,0.314183


In [6]:
# Name the population CSV file
population_file = pathlib.Path('population_total_by_country.csv')

# Read and display the population CSV with Pandas
population_df = pd.read_csv(population_file)

In [7]:
population_df_2 = population_df[['Country Name', 'Country Code', 'Indicator Name', '2006', '2007','2008','2009','2010','2011','2012','2013','2014','2015','2016']]
population_df_2

,Country Name,Country Code,Indicator Name,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,"Population, total",100834.0,101222.0,101358.0,101455.0,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0
1,Afghanistan,AFG,"Population, total",26433049.0,27100536.0,27722276.0,28394813.0,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0
2,Angola,AGO,"Population, total",20149901.0,20905363.0,21695634.0,22514281.0,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0
3,Albania,ALB,"Population, total",2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0
4,Andorra,AND,"Population, total",80993.0,82684.0,83862.0,84463.0,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,"Population, total",1719536.0,1733404.0,1747383.0,1761474.0,1775680.0,1791000.0,1807106.0,1818117.0,1812771.0,1788196.0,1777557.0
260,"Yemen, Rep.",YEM,"Population, total",20687646.0,21282515.0,21892146.0,22516460.0,23154855.0,23807588.0,24473178.0,25147109.0,25823485.0,26497889.0,27168210.0
261,South Africa,ZAF,"Population, total",48489459.0,49119759.0,49779471.0,50477011.0,51216964.0,52004172.0,52834005.0,53689236.0,54545991.0,55386367.0,56203654.0
262,Zambia,ZMB,"Population, total",12173514.0,12502958.0,12848530.0,13215139.0,13605984.0,14023193.0,14465121.0,14926504.0,15399753.0,15879361.0,16363507.0


In [8]:
# Merge the above data frames to a single data frame
merged_df = pd.concat([CO2_emission_df_2, population_df_2], keys=['CO2_emission', 'population'])
#           names=['Country Name', 'Row ID'])
merged_df

Country Name Country Code  \
CO2_emission 0           Aruba          ABW   
             1     Afghanistan          AFG   
             2          Angola          AGO   
             3         Albania          ALB   
             4         Andorra          AND   
...                        ...          ...   
population   259        Kosovo          XKX   
             260   Yemen, Rep.          YEM   
             261  South Africa          ZAF   
             262        Zambia          ZMB   
             263      Zimbabwe          ZWE   

                                          Indicator Name          2006  \
CO2_emission 0    CO2 emissions (metric tons per capita)  2.694773e+01   
             1    CO2 emissions (metric tons per capita)  6.242753e-02   
             2    CO2 emissions (metric tons per capita)  1.105019e+00   
             3    CO2 emissions (metric tons per capita)  1.302576e+00   
             4    CO2 emissions (metric tons per capita)  6.746052e+00   
...                                                  ...           ...   
population   259                       Population, total  1.719536e+06   
             260                       Population, total  2.068765e+07   
             261                       Population, total  4.848946e+07   
             262                       Population, total  1.217351e+07   
             263                       Population, total  1.215549e+07   

                          2007          2008          2009          2010  \
CO2_emission 0    2.789502e+01  2.622955e+01  2.591532e+01  2.467053e+01   
             1    8.389281e-02  1.517209e-01  2.383985e-01  2.899876e-01   
             2    1.203134e+00  1.185000e+00  1.234425e+00  1.244092e+00   
             3    1.322335e+00  1.484311e+00  1.495600e+00  1.578574e+00   
             4    6.519387e+00  6.427810e+00  6.121580e+00  6.122595e+00   
...                        ...           ...           ...           ...   
population   259  1.733404e+06  1.747383e+06  1.761474e+06  1.775680e+06   
             260  2.128252e+07  2.189215e+07  2.251646e+07  2.315486e+07   
             261  4.911976e+07  4.977947e+07  5.047701e+07  5.121696e+07   
             262  1.250296e+07  1.284853e+07  1.321514e+07  1.360598e+07   
             263  1.225592e+07  1.237955e+07  1.252697e+07  1.269772e+07   

                          2011          2012          2013          2014  \
CO2_emission 0    2.450752e+01  1.315772e+01  8.353561e+00  8.410064e+00   
             1    4.064242e-01  3.451488e-01  2.804546e-01  2.537279e-01   
             2    1.262825e+00  1.361181e+00  1.295075e+00  1.664741e+00   
             3    1.803715e+00  1.697966e+00  1.697279e+00  1.900070e+00   
             4    5.867410e+00  5.916884e+00  5.901775e+00  5.832906e+00   
...                        ...           ...           ...           ...   
population   259  1.791000e+06  1.807106e+06  1.818117e+06  1.812771e+06   
             260  2.380759e+07  2.447318e+07  2.514711e+07  2.582348e+07   
             261  5.200417e+07  5.283400e+07  5.368924e+07  5.454599e+07   
             262  1.402319e+07  1.446512e+07  1.492650e+07  1.539975e+07   
             263  1.289432e+07  1.311513e+07  1.335036e+07  1.358668e+07   

                          2015          2016  
CO2_emission 0    8.610374e+00  8.426911e+00  
             1    2.625557e-01  2.451014e-01  
             2    1.240245e+00  1.202861e+00  
             3    1.602648e+00  1.577163e+00  
             4    5.969786e+00  6.072370e+00  
...                        ...           ...  
population   259  1.788196e+06  1.777557e+06  
             260  2.649789e+07  2.716821e+07  
             261  5.538637e+07  5.620365e+07  
             262  1.587936e+07  1.636351e+07  
             263  1.381463e+07  1.403039e+07  

[528 rows x 14 columns]

In [10]:
merged_df.to_csv('output.csv')

In [11]:
# merged_table = merged_df.to_html().replace('\n','')
# merged_table

In [12]:
# Create a data frame to show the TOTAL carbon emission per country

# Drop string columns in the data frames
CO2_emission_df_value = CO2_emission_df_2.drop(columns=['Country Name', 'Country Code', 'Indicator Name'])
CO2_emission_df_value

population_df_value = population_df_2.drop(columns=['Country Name', 'Country Code', 'Indicator Name'])
population_df_value

# Multiply the data frames
total_emission_value = CO2_emission_df_value * population_df_value.values
total_emission_value

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,2.717247e+06,2.823590e+06,2.658575e+06,2.629239e+06,2.508228e+06,2.500894e+06,1.349456e+06,8.617450e+05,8.727460e+05,8.984150e+05,8.837470e+05
1,1.650150e+06,2.273540e+06,4.206049e+06,6.769282e+06,8.463436e+06,1.224045e+07,1.075531e+07,9.050156e+06,8.467103e+06,9.035488e+06,8.672455e+06
2,2.226602e+07,2.515195e+07,2.570934e+07,2.779219e+07,2.905731e+07,3.058645e+07,3.417644e+07,3.369240e+07,4.485108e+07,3.458348e+07,3.469349e+07
3,3.898021e+06,3.927357e+06,4.374731e+06,4.378398e+06,4.598418e+06,5.240143e+06,4.924781e+06,4.913780e+06,5.489499e+06,4.616753e+06,4.536079e+06
4,5.463830e+05,5.390490e+05,5.390490e+05,5.170470e+05,5.170470e+05,4.913780e+05,4.877110e+05,4.767100e+05,4.620420e+05,4.657090e+05,4.693760e+05
...,...,...,...,...,...,...,...,...,...,...,...
259,NaN,NaN,7.455011e+06,8.192078e+06,8.426766e+06,8.368094e+06,7.942722e+06,8.074734e+06,7.260660e+06,8.408431e+06,8.943813e+06
260,2.131627e+07,2.097157e+07,2.233936e+07,2.456523e+07,2.343213e+07,2.061221e+07,1.869803e+07,2.549298e+07,2.466791e+07,1.317186e+07,1.060863e+07
261,4.478947e+08,4.665781e+08,4.967722e+08,5.031124e+08,4.698160e+08,4.773701e+08,4.646749e+08,4.592221e+08,4.844950e+08,4.639525e+08,4.766440e+08
262,2.291875e+06,1.928842e+06,2.170864e+06,2.508228e+06,2.691578e+06,2.922599e+06,3.678001e+06,3.956693e+06,4.503076e+06,4.532412e+06,5.141134e+06


In [13]:
# Save the total emission dataframe to a CSV
country_names = CO2_emission_df_2[['Country Name', 'Country Code']]
total_emission_df = country_names.join(total_emission_value)

total_emission_df.to_csv('total_emission_by_country.csv',index=False)

### Find Goolge Trends for Keywords searches ("Sustainability", "Environment", "Climate Change" and "Clean Energy") by regions. The result will be used for visualization.

In [14]:
# Find the interest by region for the defined keywords
kw_list = ["Sustainability", "Environment", "Climate Change", "Clean Energy"] 
pytrend.build_payload(kw_list, timeframe='2000-01-01 2017-01-01')
# Interest by Region
interest_by_region = pytrend.interest_by_region()
interest_by_region

,Sustainability,Environment,Climate Change,Clean Energy
geoName,,,,
Afghanistan,0,0,0,0
Albania,0,0,0,0
Algeria,0,0,0,0
American Samoa,0,0,0,0
Andorra,0,0,0,0
...,...,...,...,...
Western Sahara,0,0,0,0
Yemen,0,0,0,0
Zambia,0,0,0,0


### Find the popularities of keywords search ("Sustainability", "Environment", "Climate Change" and "Clean Energy") over time. The results will be used for visualization. 

In [15]:
# Save the interest_by_region data frame to a CSV file
interest_by_region.to_csv('pytrend_keywords_by_country.csv')

In [16]:
# Find keyword rankings from U.S. trend
kw_list = ["Sustainability", "Environment", "Climate Change", "Clean Energy"] 
pytrend.build_payload(kw_list, timeframe='2000-01-01 2017-01-01',geo='US')

# Find the search volume of each keyword over time, and save the result in a data frame
search_trend_US = pytrend.interest_over_time()
search_trend_US

,Sustainability,Environment,Climate Change,Clean Energy,isPartial
date,,,,,
2004-01-01,5,85,3,1,False
2004-02-01,5,86,8,1,False
2004-03-01,4,95,4,1,False
2004-04-01,5,100,6,1,False
2004-05-01,6,86,5,1,False
...,...,...,...,...,...
2016-09-01,9,53,14,2,False
2016-10-01,8,47,17,2,False
2016-11-01,8,49,30,2,False


In [17]:
search_trend_US_df = search_trend_US.groupby(search_trend_US.index.map(lambda x: x.year)).sum()
search_trend_US_df_2 = search_trend_US_df[['Sustainability','Environment','Climate Change','Clean Energy']]
search_trend_US_df_2

,Sustainability,Environment,Climate Change,Clean Energy
date,,,,
2004,53,981,55,13
2005,53,853,52,18
2006,55,690,65,21
2007,77,681,122,25
2008,100,681,122,33
2009,113,668,157,43
2010,117,629,114,34
2011,112,551,91,30
2012,106,521,93,29


In [18]:
# Save to a CSV
search_trend_US_df_2.to_csv('keyword_search_volume_US.csv')

In [19]:
# # Calculate the average for each column in the data frame
# avg_popularity = []
# for item in kw_list:
#     avg_popularity.append(search_trend_US[item].mean())

# avg_popularity

In [20]:
# # Store avg info in a dictionary
# avg_popularity_dict = {
#     'Sustainability': avg_popularity[0],
#     'Environment': avg_popularity[1],
#     'Climate Change': avg_popularity[2],
#     'Clean Energy': avg_popularity[3]
# }

# # Convert the dictionary to a data frame
# avg_popularity_US = pd.DataFrame(data=avg_popularity_dict, index=[0])
# avg_popularity_US


### Find Google query trends for "Sustainability", "Environment", "Climate Change" and "Clean Energy". The result will be converted to HTML table strings for final web applications.

In [21]:
pytrend.build_payload(kw_list = ["Sustainability", "Environment", "Climate Change", "Clean Energy"] )
# Related Queries, returns a dictionary of dataframes
related_queries = pytrend.related_queries()
# related_queries.values()
print(related_queries)

{'Sustainability': {'top':                            query  value
0   environmental sustainability    100
1                  environmental    100
2         what is sustainability     87
3                    sustainable     85
4      sustainability definition     80
5            sustainability jobs     56
6          sustainability report     50
7          energy sustainability     43
8        business sustainability     40
9      sustainability management     34
10         social sustainability     33
11    sustainability environment     32
12          water sustainability     32
13                   environment     31
14         global sustainability     30
15      what does sustainability     30
16        sustainability science     30
17           food sustainability     29
18  definition of sustainability     28
19         google sustainability     27
20      corporate sustainability     24
21      office of sustainability     20
22       sustainability examples     19
23         de

In [22]:
# # build list of keywords
# kw_list = ["Sustainability", "Environment", "Climate Change", "Clean Energy"] 
# # build the payload
# pytrend.build_payload(kw_list, timeframe='2000-01-01 2017-01-01')
# interest_over_time = pytrend.interest_over_time()
# interest_over_time_html = interest_over_time.to_html().replace('\n','')
# # print(interest_over_time_html)

### Grab world countries capital cities and their locations

In [111]:
# Name the capitals CSV file
capitals_file = pathlib.Path('country-capitals.csv')

# Read and display the CO2 Emission CSV with Pandas
capitals_df = pd.read_csv(capitals_file)
capitals_df = capitals_df.drop(columns=['CountryCode','ContinentName','CapitalName'])
# capitals_df = capitals_df.rename(columns = {'CountryName':'Country Name'})

In [166]:
CO2_emission_df_3 = CO2_emission_df_2.rename(columns = {'Country Name':'CountryName'})
CO2_emission_df_3['avgCarbonEmission'] = round(CO2_emission_df_3.mean(axis=1),2)
CO2_emission_df_3

,CountryName,Country Code,Indicator Name,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,avgCarbonEmission
0,Aruba,ABW,CO2 emissions (metric tons per capita),26.947726,27.895023,26.229553,25.915322,24.670529,24.507516,13.157722,8.353561,8.410064,8.610374,8.426911,18.47
1,Afghanistan,AFG,CO2 emissions (metric tons per capita),0.062428,0.083893,0.151721,0.238399,0.289988,0.406424,0.345149,0.280455,0.253728,0.262556,0.245101,0.24
2,Angola,AGO,CO2 emissions (metric tons per capita),1.105019,1.203134,1.185000,1.234425,1.244092,1.262825,1.361181,1.295075,1.664741,1.240245,1.202861,1.27
3,Albania,ALB,CO2 emissions (metric tons per capita),1.302576,1.322335,1.484311,1.495600,1.578574,1.803715,1.697966,1.697279,1.900070,1.602648,1.577163,1.59
4,Andorra,AND,CO2 emissions (metric tons per capita),6.746052,6.519387,6.427810,6.121580,6.122595,5.867410,5.916884,5.901775,5.832906,5.969786,6.072370,6.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,CO2 emissions (metric tons per capita),NaN,NaN,4.266386,4.650695,4.745656,4.672303,4.395272,4.441262,4.005283,4.702186,5.031520,4.55
260,"Yemen, Rep.",YEM,CO2 emissions (metric tons per capita),1.030386,0.985390,1.020428,1.090990,1.011975,0.865783,0.764021,1.013754,0.955251,0.497091,0.390480,0.88
261,South Africa,ZAF,CO2 emissions (metric tons per capita),9.236950,9.498786,9.979458,9.967159,9.173055,9.179457,8.794997,8.553336,8.882322,8.376655,8.480658,9.10
262,Zambia,ZMB,CO2 emissions (metric tons per capita),0.188267,0.154271,0.168958,0.189800,0.197823,0.208412,0.254267,0.265078,0.292412,0.285428,0.314183,0.23


In [182]:
population_df_3 = population_df_2.rename(columns = {'Country Name':'CountryName'})
population_df_3['avgPopulation'] = round(population_df_3.mean(axis=1),0)
population_df_3

,CountryName,Country Code,Indicator Name,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,avgPopulation
0,Aruba,ABW,"Population, total",100834.0,101222.0,101358.0,101455.0,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,102481.0
1,Afghanistan,AFG,"Population, total",26433049.0,27100536.0,27722276.0,28394813.0,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,30504735.0
2,Angola,AGO,"Population, total",20149901.0,20905363.0,21695634.0,22514281.0,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,24330404.0
3,Albania,ALB,"Population, total",2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2917910.0
4,Andorra,AND,"Population, total",80993.0,82684.0,83862.0,84463.0,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,81629.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,"Population, total",1719536.0,1733404.0,1747383.0,1761474.0,1775680.0,1791000.0,1807106.0,1818117.0,1812771.0,1788196.0,1777557.0,1775657.0
260,"Yemen, Rep.",YEM,"Population, total",20687646.0,21282515.0,21892146.0,22516460.0,23154855.0,23807588.0,24473178.0,25147109.0,25823485.0,26497889.0,27168210.0,23859189.0
261,South Africa,ZAF,"Population, total",48489459.0,49119759.0,49779471.0,50477011.0,51216964.0,52004172.0,52834005.0,53689236.0,54545991.0,55386367.0,56203654.0,52158735.0
262,Zambia,ZMB,"Population, total",12173514.0,12502958.0,12848530.0,13215139.0,13605984.0,14023193.0,14465121.0,14926504.0,15399753.0,15879361.0,16363507.0,14127597.0


In [183]:
loc_emission_df = capitals_df.merge(CO2_emission_df_3, on='CountryName')
loc_emission_df = loc_emission_df.drop(columns=['Country Code','Indicator Name','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016'])
loc_emission_pop_df = loc_emission_df.merge(population_df_3, on='CountryName')
loc_emission_pop_df = loc_emission_pop_df.drop(columns=['Country Code','Indicator Name','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016'])
# loc_emission_df = loc_emission_df.set_index('Country Name')
# loc_emission_df = loc_emission_df.reset_index(drop=True)
# loc_emission_df = loc_emission_df.transpose()
# loc_emission_df = loc_emission_df.rename(columns = {'2014':'CarbonEmission'}).dropna()
loc_emission_pop_df

,CountryName,CapitalLatitude,CapitalLongitude,avgCarbonEmission,avgPopulation
0,Nauru,-0.547700,166.920867,4.12,10731.0
1,Afghanistan,34.516667,69.183333,0.24,30504735.0
2,Albania,41.316667,19.816667,1.59,2917910.0
3,Algeria,36.750000,3.050000,3.43,36839863.0
4,American Samoa,-14.266667,-170.700000,NaN,56564.0
...,...,...,...,...,...
185,Vietnam,21.033333,105.850000,1.61,88981669.0
186,Zambia,-15.416667,28.283333,0.23,14127597.0
187,Zimbabwe,-17.816667,31.033333,0.74,12982469.0
188,"Hong Kong SAR, China",22.319300,114.169400,6.08,7089655.0


In [187]:
loc_emission_pop_df = loc_emission_pop_df.rename(columns = {'avgCarbonEmission':'CarbonEmission', 'avgPopulation':'Population'}).dropna()
loc_emission_pop_df['Population'] = loc_emission_pop_df['Population'].astype(int)
loc_emission_pop_df

,CountryName,CapitalLatitude,CapitalLongitude,CarbonEmission,Population
0,Nauru,-0.547700,166.920867,4.12,10731
1,Afghanistan,34.516667,69.183333,0.24,30504735
2,Albania,41.316667,19.816667,1.59,2917910
3,Algeria,36.750000,3.050000,3.43,36839863
5,Andorra,42.500000,1.516667,6.14,81629
...,...,...,...,...,...
185,Vietnam,21.033333,105.850000,1.61,88981669
186,Zambia,-15.416667,28.283333,0.23,14127597
187,Zimbabwe,-17.816667,31.033333,0.74,12982469
188,"Hong Kong SAR, China",22.319300,114.169400,6.08,7089655


In [188]:
# loc_emission_df.to_json('emission_by_location.json')
loc_emission_pop_df.to_json('emission_population_location_copy.json',orient='records')

In [47]:
# loc_emission_json = loc_emission_df.to_json()
# loc_emission_json

# loc_emission_json = json.dumps(loc_emission_json, sort_keys=True)
# loc_emission_json
# emission_by_loc = open('emission_by_location.json','w')
# emission_by_loc.write(loc_emission_json)

In [175]:
total_emission_df['average'] = total_emission_df.mean(axis=1)

In [176]:
# total_emission_df.sort_values(by=['Country Name'],ascending=True)
# total_emission_df['Country Name'].unique()
# country_filter = ['High income', 'Low income', 'Middle income', 'Upper middle income','Lower middle income' ]

emissions_by_income = total_emission_df.loc[(total_emission_df['Country Name']=='High income')|(total_emission_df['Country Name']=='Low income')|(total_emission_df['Country Name']=='Middle income')|(total_emission_df['Country Name']=='Upper middle income')|(total_emission_df['Country Name']=='Lower middle income')]
emissions_by_income = emissions_by_income.rename(columns={'Country Name':'IncomeLevel', 'average':'avgEmission'})
emissions_by_income

,IncomeLevel,Country Code,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,avgEmission,avgEmission
93,High income,HIC,1.380044e+10,1.387558e+10,1.368713e+10,1.295561e+10,1.338530e+10,1.314847e+10,1.306566e+10,1.307138e+10,1.298531e+10,1.290927e+10,1.266931e+10,1.221428e+10,1.323213e+10
134,Low income,LIC,2.017976e+08,2.076683e+08,2.220689e+08,2.063519e+08,2.053079e+08,1.901356e+08,1.779063e+08,1.707667e+08,1.789423e+08,1.645656e+08,1.682030e+08,1.756963e+08,1.903376e+08
137,Lower middle income,LMC,2.738802e+09,2.888210e+09,3.078791e+09,3.200253e+09,3.316893e+09,3.499128e+09,3.714345e+09,3.736262e+09,3.982718e+09,4.101459e+09,4.229048e+09,3.229587e+09,3.498719e+09
154,Middle income,MIC,1.500589e+10,1.541615e+10,1.675575e+10,1.673773e+10,1.831857e+10,1.973064e+10,2.041699e+10,2.058268e+10,2.091511e+10,2.094005e+10,2.094755e+10,1.726717e+10,1.870610e+10
247,Upper middle income,UMC,1.226710e+10,1.252791e+10,1.367677e+10,1.353747e+10,1.500168e+10,1.623152e+10,1.670265e+10,1.684643e+10,1.693239e+10,1.683859e+10,1.671849e+10,1.403757e+10,1.520736e+10


In [174]:
# Save to a CSV
emissions_by_income.to_csv('emissions_by_income_level_copy.csv')